# Fantasy Football

In this notebook, I will make an attempt to predict football stats for the NFL player Tom Brady.
These stats could be used in a fantasy football league.

This notebook builds on the previous examples, but I will be using a different approach for making predictions.

To access NFL data, I will use the NFLGame module (only using pre-2018 data as of this writing).

- Original repository (pre-2018): https://github.com/BurntSushi/nflgame (`nflgame`)
- New repository (2018+): https://github.com/derek-adair/nflgame (`nflgame-redux`)
- API http://nflgame.derekadair.com/

My predictions will be compared with those from Yahoo! and ESPN, and ultimately graded on the actual outcome in each game.

For this approach, I will use play-level data to make predictions.  
There are plenty of plays per game, but not enough games in a year to really make strong predictions (at least, this was the case in my previous approach, perhaps there is a better way!).

Thus, I will use the following method:

- Categorize plays based on 
    1. Game Half (1st or 2nd)
    2. Score Differential (<= 7 pts, >7 pts; positive for 'leading', negative for 'deficit')
    3. Down (1, 2, 3 -- ignoring 4th for now)
    4. Distance (<=5; >5 yds)
    5. Field position ( [0-25], [25-50]; (+) for opponent territory, (-) for own territory)
- For each play, identify the category then register the number of [`passing_yds`, `passing_tds`, etc.] in plays that feature TB12 and plays featuring the opposing defense.
    - This will yield two distributions: 
        1. `passing_yds` for TB12 per play
        2. `passing_yds` allowed by DEF per play
- Weight the two predictions (TB12 and DEF) and predict the total number of `passing_yds` for TB12 against DEF in the upcoming game (need to figure out the weights for the average) and multiply by the number of expected plays for each category (need to predict the expected number of plays from each category -- possibly create an ensemble by drawing from the distribution of all games involving TB12 and the DEF).

## Getting Started

First we will install `nflgame-redux` (it is Python2 only!) and start playing around with the API to access individual plays for TB12 and the "BUF" defense

In [1]:
# install the nflgame module
!pip install nflgame-redux

    100% |████████████████████████████████| 42.8MB 470kB/s 
    100% |████████████████████████████████| 5.8MB 4.2MB/s 
  Running setup.py bdist_wheel for nflgame-redux ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/ae/fd/75/e7cda3251d7c91489cac6a73daff6b6c468c8d579a76746ead
Successfully built nflgame-redux


In [1]:
import nflgame

In [2]:
# Investigate single player
tb12 = nflgame.find("Tom Brady")[0]
print tb12.player_id

00-0019596


In [3]:
# copied from notebook 0:
def get_tb12_stats(player,stat_names,week):
    """Player Weekly stats"""
    stat_values = {}
    stats = player.stats(ffyear,week=week)

    for stat_name in stat_names:
        try:
            stat_values[stat_name] = getattr(stats,stat_name)()
        except:
            stat_values[stat_name] = getattr(stats,stat_name)

    return stat_values

In [4]:
# Investigate plays for a player (season and week)
plays = tb12.plays(2013,week=1)

In [5]:
# Here is what a typical play might look like
for i,p in enumerate(plays):
    print i,p
    print dir(p)
    for event in p.events:
        print event
    if i==1: break

0 (NE, NE 19, Q1, 1 and 5) (14:56) T.Brady pass short right to D.Amendola to NE 29 for 10 yards (J.Rogers). Slant pattern, caught at NE 25, crossing to middle.
['_Play__players', '__class__', '__delattr__', '__dict__', '__doc__', '__eq__', '__format__', '__getattr__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_stats', 'data', 'defense_tkl', 'desc', 'down', 'drive', 'events', 'first_down', 'has_player', 'home', 'note', 'passing_att', 'passing_cmp', 'passing_cmp_air_yds', 'passing_first_down', 'passing_yds', 'players', 'playid', 'receiving_rec', 'receiving_tar', 'receiving_yac_yds', 'receiving_yds', 'team', 'time', 'touchdown', 'yardline', 'yards_togo']
{'receiving_yac_yds': 4, 'playerid': u'00-0026035', 'playername': u'D.Amendola', 'team': u'NE'}
{'playerid': u'00-0028144', 'playername': u'J.Rogers', 'defense_tkl': 1, 'team': u'BUF'}
{'passing

Now, for each play we will run through the `playerid`s, select those that belong to TB12, and categorize the play and result.

For now, let's just consider `passing_yds` as our target value.

In [6]:
plays = list(plays)

Relevant attributes:
- `time`: Attributes `clock` and `qtr`
- `yards_togo`: Just use the value reported
- `yardline`: Attribute `offset` >0 for opponent side of field, negative for own side of filed.  Represents difference from 50 yard line.  
    - `if yardline.offset < 0: # OWN`
    - `else: # opponent`
    - `50 - offset = yardline`
- `down`: Just use the value reported

In [7]:
games = nflgame.games_gen(2013,1,"NE","NE")
for i,g in enumerate(games):
    print i,g.scores[0]

0 NE - Q1 - TD - J.Edelman 9 yd. pass from T.Brady (S.Gostkowski kick is good) Drive: 3 plays, 16 yards in 0:47


In [8]:
# Here is what a typical play might look like
# Investigate plays for a player (season and week)
plays = tb12.plays(2013,week=1)

for i,p in enumerate(plays):
    if 'TOUCHDOWN' in p.desc: print i,p.desc

6 (10:51) (Shotgun) T.Brady pass short middle to J.Edelman for 9 yards, TOUCHDOWN. Caught in back of end zone.
24 (2:00) (Shotgun) T.Brady pass short middle to J.Edelman for 8 yards, TOUCHDOWN. Caught 6 yds. into end zone, crossing from right.


At the moment it seems like there will be a major hack necessary to line up which plays go with which scores to get the live score at any given play (since the individual plays don't report the current score).  Thus, I will skip that feature for now, but will come back to it if I think it can add something signficant (the current score certainly affects play calling!).